## Library imports

In [1]:
##!pip install pickle5

In [2]:
import os
import gc
import csv
import glob
import pickle
import random
import collections
import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle5 as pickle
from pathlib import Path
from dataclasses import dataclass
from typing import List, Tuple, Any

## for spline interpolation and evaluation
from scipy.interpolate import splev, splrep

## for ESEKF imports
import sys
sys.path.append('ESEKF/')
from rotations import Quaternion, skew_symmetric

## plotting library
import matplotlib.pyplot as plt
import seaborn as sns

import dask
import multiprocessing
from dask.distributed import wait
from dask.distributed import Client, wait, LocalCluster

In [3]:
# set n_workers to number of cores
## client = Client(n_workers=multiprocessing.cpu_count(), threads_per_worker=1)
## client

## Configuration parameters

In [4]:
floor_map = {"B2": -2, "B1": -1, "F1": 0, "F2": 1, "F3": 2, "F4": 3, "F5": 4, "F6": 5, "F7": 6, "F8": 7, "F9": 8,
             "1F": 0, "2F": 1, "3F": 2, "4F": 3, "5F": 4, "6F": 5, "7F": 6, "8F": 7, "9F": 8}

sampleCsvPath = 'sample_submission.csv'
waypointData_trainPath = 'wayPointData_train.pickle'
ssubm = pd.read_csv(sampleCsvPath)
ssubm_df = ssubm["site_path_timestamp"].apply(lambda x: pd.Series(x.split("_")))

outputDir = '.'
featuresInputPath = 'referencePublicNotebooks/imuFeatures'
pictureSaveDir = 'referencePublicNotebooks/splineFitOutput'
ACC_COLS  = ['ts', 'ax', 'ay', 'az', 'a_acc']
GYRO_COLS = ['ts', 'gx', 'gy', 'gz', 'g_acc']
AHRS_COLS = ['ts', 'qx', 'qy', 'qz', 'q_acc']

## exp weighted moving avg parameter
smoothSpan = 10

## gravity vector to calculte linear accelearation
gravity = np.array([0.0, 0.0, -9.8])

## number of time sequences to give as input to encoder
imuInputSequenceLength = 100

## max number of time sequences in decoder
wayPointMaxSequenceLength = 107

## Helper functions

In [5]:
def input_dir() -> Path:
    return Path('.')
    #return Path('.')

def generate_target_buildings() -> List[str]:
    ssubm = pd.read_csv(sampleCsvPath)
    ssubm_df = ssubm["site_path_timestamp"].apply(
        lambda x: pd.Series(x.split("_")))
    buildingsList = sorted(ssubm_df[0].value_counts().index.tolist()) # type: ignore
    return buildingsList

In [6]:
def getBuildingPathFiles_test(building):
    pathFilesTest = list(set(sorted(ssubm_df[ssubm_df[0] == building][1].values.tolist())))
    buildingPathFilesTest = [f"{input_dir()}/test/{path}.txt" for path in pathFilesTest]
    return buildingPathFilesTest

In [7]:
def getWayPointData_train():
    with open(waypointData_trainPath,'rb') as inputFile:
        waypointData_train = pickle.load(inputFile)
    return waypointData_train

def getWayPointCount():
    wayPointData_train = getWayPointData_train()
    buildingList = sorted(wayPointData_train.building.unique().tolist())
    pathList     = sorted(wayPointData_train.path.unique().tolist())

    output = []
    wayPointBins = [0,5,10,20,84,110]
    for building, buildingData in wayPointData_train.groupby(by='building'):
        for path, pathData in buildingData.groupby(by='path'):
            output.append([building, path, pathData.shape[0]])

    output = pd.DataFrame(output, columns =['building', 'path', 'count'])  
    output['countBin'] = pd.cut(output['count'], bins=wayPointBins)
    output.to_pickle('wayPoint_count.pickle')
    return output

In [8]:
def extract_IMUData(pathFile):
    acce, gyro, ahrs = [], [], []
    with open(pathFile) as f:
        for line_data in csv.reader(f, delimiter="\t", doublequote=True):
            if line_data[1] == 'TYPE_ACCELEROMETER':
                if len(line_data) > 5:
                    accuracy = np.int16(line_data[-1])
                else:
                    accuracy = np.int16(3)
                acce.append([np.int64(line_data[0]), np.float32(line_data[2]), np.float32(line_data[3]), np.float32(line_data[4]), accuracy])
                continue

            elif line_data[1] == 'TYPE_GYROSCOPE':
                if len(line_data) > 5:
                    accuracy = np.int16(line_data[-1])
                else:
                    accuracy = np.int16(3)               
                gyro.append([np.int64(line_data[0]), np.float32(line_data[2]), np.float32(line_data[3]), np.float32(line_data[4]), accuracy])
                continue

            if line_data[1] == 'TYPE_ROTATION_VECTOR':
                if len(line_data)>5:
                    accuracy = np.int16(line_data[-1])
                else:
                    accuracy = np.int16(3)
                if len(line_data)>=5:        
                    ahrs.append([np.int64(line_data[0]), np.float32(line_data[2]), np.float32(line_data[3]), np.float32(line_data[4]), accuracy])
                continue

    ## sort data by timestamps
    acce = sorted(acce, key=lambda x: x[0])
    gyro = sorted(gyro, key=lambda x: x[0])
    ahrs = sorted(ahrs, key=lambda x: x[0])
    
    acce = pd.DataFrame(acce, columns = ACC_COLS)
    gyro = pd.DataFrame(gyro, columns = GYRO_COLS)
    ahrs = pd.DataFrame(ahrs, columns = AHRS_COLS)
    return acce, gyro, ahrs

In [9]:
def get_qw(qx,qy,qz):
    qw = 0.0
    temp = 1 - (qx**2 + qy**2 + qz**2)
    if temp > 0.0:
        qw = np.sqrt(temp)
    return qw
    

def convertQuat(ahrsData):
    numRows = ahrsData.shape[0]
    rotMatList= []
    rollList, pitchList, yawList = [], [], []    
    for row in range(numRows):
        quat = Quaternion(w=ahrsData['qw'][row], x=ahrsData['qx'][row],\
                          y=ahrsData['qy'][row], z=ahrsData['qz'][row])
        eulerAngles = np.float64(quat.to_euler())
        
        ## add to output
        rotMatList.append(np.float64(quat.to_mat()))
        rollList.append(eulerAngles[0])
        pitchList.append(eulerAngles[1])
        yawList.append(eulerAngles[2])
    return rotMatList, rollList, pitchList, yawList

def processAHRSData(ahrs):
    ahrs['qw'] = ahrs.apply(lambda row : get_qw(row['qx'], row['qy'], row['qz']), axis=1) 
    ahrs['rotMat'], ahrs['roll'], ahrs['pitch'], ahrs['yaw'] = convertQuat(ahrs)
    ahrs = ahrs.drop(columns=['qw', 'qx', 'qy', 'qz'])
    return ahrs

In [10]:
def acceVector(ax,ay,az):
    return np.array([ax,ay,az])

def getLinearAccFromRawAcc(rotMatrix, rawAcc):
    numRows = rotMatrix.shape[0]
    linAcc_x, linAcc_y = [], []
    for row in range(numRows):
        linearAcceleration = (rotMatrix[row] @ rawAcc[row]) + gravity
        linAcc_x.append(linearAcceleration[0])
        linAcc_y.append(linearAcceleration[1])
    return linAcc_x, linAcc_y

def processAcceData(acceData, ahrsData):
    acceData['ax_s'] = acceData['ax'].ewm(span=smoothSpan, adjust=True).mean()
    acceData['ay_s'] = acceData['ay'].ewm(span=smoothSpan, adjust=True).mean()
    acceData['az_s'] = acceData['az'].ewm(span=smoothSpan, adjust=True).mean()
    acceData['acc'] = acceData.apply(lambda row : acceVector(row['ax_s'], row['ay_s'], row['az_s']), axis=1)
    acceData['lin_ax'], acceData['lin_ay'] = getLinearAccFromRawAcc(ahrsData['rotMat'], acceData['acc'])
    return acceData

In [11]:
def processGyroData(gyroData):
    gyroData['gz_s'] = gyroData['gz'].ewm(span=smoothSpan, adjust=True).mean()
    return gyroData

In [12]:
def getIMUData(acceData, gyroData, ahrsData):
    ahrsData = processAHRSData(ahrsData);
    acceData = processAcceData(acceData, ahrsData);
    gyroData = processGyroData(gyroData);
    imuData = pd.concat([acceData[['ts','lin_ax', 'lin_ay']], gyroData[['gz_s']], \
                         ahrsData[['roll', 'pitch', 'yaw']]], axis=1);
    return imuData

In [13]:
def getPathImuInput(imuData):
    pathInitialTime = imuData['ts'].values[0]
    ## 7 rows and 100 columns, each row represents each feature
    pathImuInput = np.zeros((7, imuInputSequenceLength))
    
    ## calculating sampling frequency for path
    numRowsInPath = imuData.shape[0]
    samplingFactor = int(np.floor_divide(numRowsInPath, imuInputSequenceLength))

    ## unix time to seconds
    imuData['ts'] = (imuData['ts'] - pathInitialTime) / 1000.0    

    ## new sampled timestamps
    newTs = imuData['ts'].values[::samplingFactor][0:imuInputSequenceLength]
    pathImuInput[0] = newTs
    
    ## fitting spline for each feature
    for index,col in enumerate(['lin_ax', 'lin_ay', 'gz_s', 'roll', 'pitch', 'yaw']):
        spl = splrep(imuData['ts'].values, imuData[col].values)
        fitSpline = splev(newTs, spl)
        pathImuInput[index+1] = fitSpline
        
    return pathImuInput, pathInitialTime

In [14]:
def plotSplineFitOutput(pathName, imuData, pathImuInput, saveFig=False):
    fig, axes = plt.subplots(nrows=6, ncols=1, figsize=(20, 20))
    for index,col in enumerate(['lin_ax', 'lin_ay', 'gz_s', 'roll', 'pitch', 'yaw']):
        axes[index].plot(imuData['ts'], imuData[col], label=f"orig_{col}")
        axes[index].plot(pathImuInput[0], pathImuInput[index+1], label=f"fit_{col}")
        axes[index].legend(loc='best')
    if saveFig == True:
        plt.savefig(f"{pictureSaveDir}/{pathName}_splineFitOutput.png", dpi=200)

In [15]:
def getDecoderData(pathName, floorString):
    ## output variables
    decoderTs = np.zeros(wayPointMaxSequenceLength)
    wayPointOutput = np.zeros((wayPointMaxSequenceLength, 3))

    pathWayPointData = wayPointData_train[wayPointData_train.path == pathName].reset_index(drop=True)
    numWaypoints = pathWayPointData.shape[0]    

    ## get inference timestamps and store in decoderTs variable
    inferenceTs = (pathWayPointData['timestamp'].values - pathInitialTime) / 1000.0
    decoderTs[0:numWaypoints] = inferenceTs

    ## get local position information
    initialWayPoint = pathWayPointData.loc[0,['x', 'y']].values.astype(np.float64)
    localWayPoints = pathWayPointData.loc[:, ['x','y']].values.astype(np.float64) - initialWayPoint

    wayPointOutput[0:numWaypoints, 0:2] = localWayPoints
    wayPointOutput[0:numWaypoints, 2] = floor_map[folder.name]

    return decoderTs, wayPointOutput, numWaypoints

In [16]:
buildingsList = generate_target_buildings()
buildingsList = [buildingsList[0]] + buildingsList[2:]
wayPointData_train = getWayPointData_train()

In [17]:
pathNameList = []
buildingList = []
encoderDataList = []
decoderDataList = []
inferenceTsList = []
numWayPointsList = []

for index,building in enumerate(buildingsList):
    print(f"{index+1} bdg = {building} -----------------")
    building_path = input_dir() / 'train' / building
    folders = sorted(building_path.glob('*'))
    ## print(f"There are {len(list(folders))} floors in building")   
    
    ## iterate through each floor 
    for folder in folders:
        floorFiles = sorted(folder.glob("*.txt"))
        ## iterate through each path file
        for pathFile in tqdm(floorFiles):
            pathName = pathFile.name.split('.')[0]
            acceData, gyroData, ahrsData = extract_IMUData(pathFile)

            ## get encoder data from imu input
            imuData = getIMUData(acceData, gyroData, ahrsData)
            pathImuInput, pathInitialTime = getPathImuInput(imuData)
            ## plotSplineFitOutput(pathName, imuData,pathImuInput, saveFig=False) 
            
            ## get decoder data from waypoint data
            inferenceTs, decoderInput, numWayPoints = getDecoderData(pathName, folder.name)

            ## store output to list
            pathNameList.append(pathName)
            buildingList.append(building)
            encoderDataList.append(pathImuInput)
            decoderDataList.append(decoderInput)
            inferenceTsList.append(inferenceTs)
            numWayPointsList.append(numWayPoints)
    break

  0%|          | 0/109 [00:00<?, ?it/s]

1 bdg = 5a0546857ecc773753327266 -----------------


100%|██████████| 86/86 [00:43<00:00,  1.97it/s]


In [31]:
imuDataOutput = pd.DataFrame()
imuDataOutput['path'] = pathNameList
imuDataOutput['building'] = buildingList
imuDataOutput['encoderData'] = encoderDataList
imuDataOutput['decoderData'] = decoderDataList
imuDataOutput['inferenceTsList'] = inferenceTsList
imuDataOutput['numWayPoints'] = numWayPointsList
imuDataOutput.to_pickle(f"imuSeq2SeqData.pickle")

In [32]:
imuDataOutput.head()

,path,building,encoderData,decoderData,inferenceTsList,numWayPoints
0,5e15730aa280850006f3d005,5a0546857ecc773753327266,"[[0.0, 0.322, 0.645, 0.968, 1.291, 1.613, 1.93...","[[0.0, 0.0, -1.0], [1.365509033203125, 4.91879...","[-0.261, 9.859, 20.294, 31.007, 0.0, 0.0, 0.0,...",4
1,5e15730b1506f2000638fc29,5a0546857ecc773753327266,"[[0.0, 0.222, 0.444, 0.666, 0.887, 1.109, 1.33...","[[0.0, 0.0, -1.0], [6.1605682373046875, -1.404...","[-0.119, 7.921, 19.09, 0.0, 0.0, 0.0, 0.0, 0.0...",3
2,5e15730ca280850006f3d007,5a0546857ecc773753327266,"[[0.0, 0.262, 0.524, 0.787, 1.049, 1.311, 1.57...","[[0.0, 0.0, -1.0], [-6.646209716796875, 0.5278...","[-0.112, 8.541, 15.752, 23.097, 0.0, 0.0, 0.0,...",4
3,5e15730e1506f2000638fc2b,5a0546857ecc773753327266,"[[0.0, 0.363, 0.726, 1.089, 1.452, 1.816, 2.17...","[[0.0, 0.0, -1.0], [-5.328582763671875, 5.6999...","[-0.122, 8.202, 18.126, 26.743, 34.789, 0.0, 0...",5
4,5e15730f1506f2000638fc2d,5a0546857ecc773753327266,"[[0.0, 0.161, 0.323, 0.484, 0.645, 0.807, 0.96...","[[0.0, 0.0, -1.0], [0.8319854736328125, 4.2953...","[-0.114, 7.55, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...",2
